# AIML- INCAF-1 Módulo 2: Identificar origen de datos
## Ejercicio de limpieza y transformación de un conjunto de datos financieros.
### Elaborado por: Gabriel Guzmán

Se creado un programa Python el cuál utiliza diferentes tipos de limpieza y transformaión de datos financieros.


In [1]:
#Importamos librerias
import pandas as pd
import numpy as np
from datetime import datetime

In [20]:
#Generando semilla
np.random.seed(42)

#Creando datos de ejemplo
datos = {
    'fecha': pd.date_range(start='2024-06-01', periods=1000, freq='D'),
    'Importe': np.random.normal(45000, 350, 1000),
    'Comisión': np.random.normal(350, 150, 1000),
    'Producto': np.random.choice(['Prestámos', 'Cuenta Ahorro', 'Cuenta Corriente', 'DPF'], 1000),
    'Region': np.random.choice(['San Salvador Norte', 'San Salvador Sur', 'San Salvador Este', 'San Salvador Oeste'], 1000)
}

df = pd.DataFrame(datos)

# Simulamos algunos valores nulos y duplicados para el ejemplo
df.loc[0:5, 'Comisión'] = np.nan
df = pd.concat([df, df.iloc[0:3]])

#Mostrando datos originales
print('Dataframe Original:\n',df)

print('\nInformación de Columnas:\n')
df.info()

Dataframe Original:
          fecha       Importe    Comisión          Producto              Region
0   2024-06-01  45173.849954         NaN     Cuenta Ahorro  San Salvador Oeste
1   2024-06-02  44951.607495         NaN  Cuenta Corriente   San Salvador Este
2   2024-06-03  45226.690988         NaN     Cuenta Ahorro   San Salvador Este
3   2024-06-04  45533.060450         NaN  Cuenta Corriente   San Salvador Este
4   2024-06-05  44918.046319         NaN  Cuenta Corriente    San Salvador Sur
..         ...           ...         ...               ...                 ...
998 2027-02-24  44800.087354  325.539955  Cuenta Corriente    San Salvador Sur
999 2027-02-25  45200.403973  238.264603         Prestámos   San Salvador Este
0   2024-06-01  45173.849954         NaN     Cuenta Ahorro  San Salvador Oeste
1   2024-06-02  44951.607495         NaN  Cuenta Corriente   San Salvador Este
2   2024-06-03  45226.690988         NaN     Cuenta Ahorro   San Salvador Este

[1003 rows x 5 columns]

Infor

## Iniciando proceso de limpieza y transformación de datos...
### 1. Manejo de valores faltantes:

* Identifica valores nulos en columnas numéricas
* Rellena los valores faltantes usando la mediana de cada columna

In [3]:
# Creamos una copia para no modificar los datos originales
df_limpio = df.copy()

# 1. Manejo de valores faltantes
print("\n1. Tratamiento de valores faltantes:")
# Rellenamos valores faltantes en columnas numéricas con la mediana
columnas_numericas = df_limpio.select_dtypes(include=['float64', 'int64']).columns
for columna in columnas_numericas:
    if df_limpio[columna].isnull().sum() > 0:
        mediana = df_limpio[columna].median()
        #df_limpio[columna].fillna(mediana, inplace=True)
        df_limpio[columna] = df_limpio[columna].fillna(mediana)
        print(f"- Valores faltantes en '{columna}' rellenados con la mediana: {mediana}")




1. Tratamiento de valores faltantes:
- Valores faltantes en 'Comisión' rellenados con la mediana: 358.90197310587916


### 2.Eliminación de duplicados:

* Detecta y elimina registros duplicados en el conjunto de datos


In [4]:
# 2. Eliminación de duplicados
print("\n2. Eliminación de registros duplicados:")
duplicados = df_limpio.duplicated().sum()
df_limpio.drop_duplicates(inplace=True)
print(f"- Se eliminaron {duplicados} registros duplicados")



2. Eliminación de registros duplicados:
- Se eliminaron 3 registros duplicados


### 3.Estandarización de texto:

* Convierte texto a minúsculas
* Elimina espacios innecesarios
* Estandariza el formato de las columnas de texto

In [5]:
# 3. Estandarización de texto
print("\n3. Estandarización de columnas de texto:")
columnas_texto = df_limpio.select_dtypes(include=['object']).columns
for columna in columnas_texto:
    # Convertimos a minúsculas y eliminamos espacios externos
    df_limpio[columna] = df_limpio[columna].str.lower().str.strip()
    print(f"- Columna '{columna}' estandarizada a minúsculas y sin espacios externos")



3. Estandarización de columnas de texto:
- Columna 'Producto' estandarizada a minúsculas y sin espacios externos
- Columna 'Region' estandarizada a minúsculas y sin espacios externos


### 4.Tratamiento de valores atípicos (outliers):

* Identifica outliers usando el método del rango intercuartílico (IQR)
* Recorta los valores extremos a límites razonables

In [6]:
# 4. Manejo de valores atípicos (outliers)
print("\n4. Tratamiento de valores atípicos:")
for columna in columnas_numericas:
    Q1 = df_limpio[columna].quantile(0.25)
    Q3 = df_limpio[columna].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    # Identificamos outliers
    outliers = df_limpio[(df_limpio[columna] < limite_inferior) | 
                        (df_limpio[columna] > limite_superior)][columna]
    
    if len(outliers) > 0:
        # Recortamos los valores atípicos a los límites
        df_limpio[columna] = df_limpio[columna].clip(limite_inferior, limite_superior)
        print(f"- Se encontraron y trataron {len(outliers)} valores atípicos en '{columna}'")



4. Tratamiento de valores atípicos:
- Se encontraron y trataron 8 valores atípicos en 'Importe'
- Se encontraron y trataron 8 valores atípicos en 'Comisión'


### 5. Creación de nuevas características:

* Calcula nuevas métricas como el margen de beneficio
* Agrega características derivadas útiles para el análisis

In [7]:
# 5. Creación de nuevas características
print("\n5. Creación de nuevas características:")

# Ejemplo: Calcular ratios financieros
if 'Importe' in df_limpio.columns and 'Comisión' in df_limpio.columns:
    df_limpio['margen_beneficio'] = ((df_limpio['Importe'] - df_limpio['Comisión']) / 
                                    df_limpio['Importe'] * 100)
    print("- Creada nueva columna 'margen_beneficio'")


5. Creación de nuevas características:
- Creada nueva columna 'margen_beneficio'


### 6. Normalización de datos numéricos:

* Aplica normalización Min-Max a las columnas numéricas
* Escala los valores entre 0 y 1

In [8]:
# 6. Normalización de datos numéricos
print("\n6. Normalización de datos numéricos:")
for columna in columnas_numericas:
    # Aplicamos normalización Min-Max
    min_val = df_limpio[columna].min()
    max_val = df_limpio[columna].max()
    df_limpio[f'{columna}_normalizado'] = ((df_limpio[columna] - min_val) / 
                                          (max_val - min_val))
    print(f"- Columna '{columna}' normalizada entre 0 y 1")



6. Normalización de datos numéricos:
- Columna 'Importe' normalizada entre 0 y 1
- Columna 'Comisión' normalizada entre 0 y 1


### 7. Conversión de tipos de datos:

* Convierte columnas de texto a formato fecha cuando es posible
* Asegura tipos de datos apropiados para cada columna

In [15]:
# 7. Conversión de tipos de datos
print("\n7. Conversión de tipos de datos:")

#Preparando el formato de fecha
date_format = '%Y-%m-%d'

# Convertir columnas de fecha
for columna in columnas_texto:
    try:
        df_limpio[f'{columna}_fecha'] = pd.to_datetime(df_limpio[columna], format=date_format)
        print(f"- Columna '{columna}' convertida a formato fecha")
    except:
        print(f"- La Columna '{columna}' no es de tipo fecha")
        continue


7. Conversión de tipos de datos:
- La Columna 'Producto' no es de tipo fecha
- La Columna 'Region' no es de tipo fecha
